In [ ]:
!pip install PyMySQL
!pip install cryptography
!pip install pykrx
!pip install plotly --upgrade

In [ ]:
from pykrx import stock
import pymysql.cursors
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly import subplots
import plotly.offline as pyo

In [ ]:
plt.style.use('seaborn')

In [ ]:
connection = pymysql.connect(host = '3.36.133.47', # EC2 아이피
                             user = 'USER_SQLSTK',   # 데이터베이스 사용자
                             password = '1qaz2wsx', 
                             database = 'DB_SQLSTK',
                             charset = 'utf8',
                             cursorclass = pymysql.cursors.DictCursor     )
cursor = connection.cursor()

In [ ]:
# sql_to_DATAFrame Function
def mk_df(sql):
  cursor.execute(sql)
  result = cursor.fetchall()
  df = pd.DataFrame(result)
  return df

# 종목 바르손이엔에이


In [ ]:
# 바르손이앤에이  -' 035620'

na_sql =  '''
SELECT   H1.STK_CD ,H1.DT
		,H1.C_PRC ,H1.O_PRC
        ,H1.H_PRC ,H1.L_PRC
        ,H1.VOL
FROM HISTORY_DT H1
WHERE H1.STK_CD = '035620'
AND H1.DT BETWEEN '2019-01-01' AND '2019-12-31'
 
 ''' 
na_df = mk_df(na_sql)
na_df

In [ ]:
na_df["DT"] = pd.to_datetime (na_df["DT"])
na_df.loc[: , "C_PRC" : ] = na_df.loc[: , "C_PRC" : ].astype(int)
# 12일 지수이동평균을 구합니다.
short = 12
na_df["ema_short"] = na_df["C_PRC"].ewm(short).mean()
# 26일 지수이동평균을 구합니다.
long = 26
na_df["ema_long"] = na_df["C_PRC"].ewm(long).mean()
# 거래일 DT를 인덱스로 설정 합니다
na_df = na_df.set_index("DT")

In [ ]:
# MACD = 단기 지수이동평균(12일 지수 이동 평균) - 장기 지수이동평균 (26일 지수 이동 평균)
na_df["MACD"] = na_df["ema_short"] - na_df["ema_long"]
# MACD의 9일 이동평균값을 구합니다.
na_df["signal"] = na_df["MACD"].ewm(9).mean()
# MACD oscillator 계산
na_df["macdhist"] = na_df["MACD"] - na_df["signal"]



In [ ]:
# 매수일
masu_df =na_df.loc[['2019-04-05','2019-05-22','2019-09-02']]

In [ ]:
mado_list = []
a = na_df.reset_index()
def find_mado(mado_day):
  mado_day = mado_day.reset_index()
  for i in mado_day["DT"]:
    filtered_df = a[a["DT"].isin(pd.date_range(i, '2019-12-30'))]
    max = 0
    for i in filtered_df["macdhist"]:
      if i > max:
        max = i
      else:
        mado_list.append(filtered_df[filtered_df["macdhist"] == i]['DT'])
        break
  return mado_list

find_mado(masu_df)

In [ ]:
# 매도일
mado_df = na_df.loc [['2019-04-22','2019-06-04','2019-09-24']]

In [ ]:
# 시가 종가 저가 고가를 나타내는 봉차트
candlestick = go.Candlestick(x=na_df.index, 
                             open=na_df.O_PRC, 
                             high=na_df.H_PRC, 
                             low=na_df.L_PRC, 
                             close=na_df.C_PRC)

# MACD 선차트
macd = go.Scatter(x=na_df.index, y=na_df["MACD"], name="MACD")

# signal 선차트
signal = go.Scatter(x=na_df.index, y=na_df["signal"], name="signal")

# macdhist 막대 차트
oscillator = go.Bar(x=na_df.index, y=na_df["macdhist"], name="oscillator")

# 시가 종가 저가 고가를 나타내는 봉차트
data = [macd, signal, oscillator]
layout = go.Layout(title = 'MACD')
fig = go.Figure(data=data,layout =layout)
fig.add_trace(
    go.Scatter(x=masu_df.index, y=masu_df["MACD"], name="매수",mode='markers', marker=dict(size=9, color='#ff9f43', symbol=18 ) ))
fig.add_trace(
    go.Scatter(x=mado_df.index, y=mado_df["MACD"], name="매도",mode='markers', marker=dict(size=9, color='#9f3f43', symbol=18 ) ))
fig.add_hline(y=1, line_dash="dot" ,annotation_text = "MACD 추세 전환 선", annotation_position = 'bottom left' )
pyo.iplot(fig)


In [ ]:
a = na_df.reset_index()

In [ ]:
filtered_df = a[a["DT"].isin(pd.date_range('2019-03-24', '2019-04-30'))]
filtered_df

In [ ]:
filtered_dfa = a[a["DT"].isin(pd.date_range('2019-05-21', '2019-06-15'))]
filtered_dfa

In [ ]:
filtered_dfb = a[a["DT"].isin(pd.date_range('2019-08-30', '2019-09-30'))]
filtered_dfb

In [ ]:
# 수익률  (sell 종가- buy 종가)   / buy 종가 * 100

buy_df =  masu_df['C_PRC']
sell_df =  mado_df['C_PRC']

In [ ]:
buy_df

DT
2019-04-05    1605
2019-05-22    2000
2019-09-02    1515
Name: C_PRC, dtype: int64

In [ ]:
sell_df
# 13,759,965
# 13.76%

DT
2019-04-22    1870
2019-06-04    2665
2019-09-24    1630
Name: C_PRC, dtype: int64

# 종목 비덴트


In [ ]:
# 비덴트  -' 121800'

b_sql =  '''
SELECT   H1.STK_CD ,H1.DT
		,H1.C_PRC ,H1.O_PRC
        ,H1.H_PRC ,H1.L_PRC
        ,H1.VOL
FROM HISTORY_DT H1
WHERE H1.STK_CD = '121800'
AND H1.DT BETWEEN '2019-01-01' AND '2019-12-31'
 
 ''' 
dent_df = mk_df(b_sql)
dent_df

In [ ]:
dent_df["DT"] = pd.to_datetime (tes_df["DT"])
dent_df.loc[: , "C_PRC" : ] = dent_df.loc[: , "C_PRC" : ].astype(int)
# 12일 지수이동평균을 구합니다.
short = 12
dent_df["ema_short"] = dent_df["C_PRC"].ewm(short).mean()
# 26일 지수이동평균을 구합니다.
long = 26
dent_df["ema_long"] = dent_df["C_PRC"].ewm(long).mean()
# 거래일 DT를 인덱스로 설정 합니다
dent_df = dent_df.set_index("DT")

In [ ]:
# MACD = 단기 지수이동평균(12일 지수 이동 평균) - 장기 지수이동평균 (26일 지수 이동 평균)
dent_df["MACD"] = dent_df["ema_short"] - dent_df["ema_long"]
# MACD의 9일 이동평균값을 구합니다.
dent_df["signal"] = dent_df["MACD"].ewm(9).mean()
# MACD oscillator 계산
dent_df["macdhist"] = dent_df["MACD"] - dent_df["signal"]



masu_df =dent_df.loc[['2019-02-22','2019-04-02','2019-05-15','2019-06-24','2019-08-13','2019-09-03','2019-10-28']]

In [ ]:
masu_df =dent_df.loc[['2019-02-22','2019-04-02','2019-05-15','2019-06-24','2019-08-13','2019-09-03','2019-10-28']]
mado_list = []
a = dent_df.reset_index()
def find_mado(mado_day):
  mado_day = mado_day.reset_index()
  for i in mado_day["DT"]:
    filtered_df = a[a["DT"].isin(pd.date_range(i, '2019-12-30'))]
    max = 0
    for i in filtered_df["macdhist"]:
      if i > max:
        max = i
      else:
        mado_list.append(filtered_df[filtered_df["macdhist"] == i]['DT'])
        break
  return mado_list

find_mado(masu_df)

In [ ]:
mado_df = tes_df.loc [['2019-02-28','2019-04-09','2019-05-17','2019-07-01','2019-08-21','2019-09-25','2019-11-08']]

In [ ]:
# 시가 종가 저가 고가를 나타내는 봉차트
candlestick = go.Candlestick(x=dent_df.index, 
                             open=dent_df.O_PRC, 
                             high=dent_df.H_PRC, 
                             low=dent_df.L_PRC, 
                             close=dent_df.C_PRC)

# MACD 선차트
macd = go.Scatter(x=dent_df.index, y=dent_df["MACD"], name="MACD")

# signal 선차트
signal = go.Scatter(x=dent_df.index, y=dent_df["signal"], name="signal")

# macdhist 막대 차트
oscillator = go.Bar(x=dent_df.index, y=dent_df["macdhist"], name="oscillator")

# 시가 종가 저가 고가를 나타내는 봉차트
data = [macd, signal, oscillator]
layout = go.Layout(title = 'MACD')
fig = go.Figure(data=data,layout =layout)
fig.add_trace(
    go.Scatter(x=masu_df.index, y=masu_df["MACD"], name="매수",mode='markers', marker=dict(size=9, color='#ff9f43', symbol=18 ) ))
fig.add_trace(
    go.Scatter(x=mado_df.index, y=mado_df["MACD"], name="매도",mode='markers', marker=dict(size=9, color='#9f3f43', symbol=18 ) ))
fig.add_hline(y=1, line_dash="dot" ,annotation_text = "MACD 추세 전환 선", annotation_position = 'bottom left' )
pyo.iplot(fig)

In [ ]:
filtered_df = a[a["DT"].isin(pd.date_range('2019-2-21', '2019-03-30'))]
filtered_df

In [ ]:
filtered_dfa = a[a["DT"].isin(pd.date_range('2019-04-02', '2019-05-01'))]
filtered_dfa

In [ ]:
filtered_dfb = a[a["DT"].isin(pd.date_range('2019-05-14', '2019-05-30'))]
filtered_dfb

In [ ]:
filtered_dfc = a[a["DT"].isin(pd.date_range('2019-06-21', '2019-07-30'))]
filtered_dfc

In [ ]:
filtered_dfc = a[a["DT"].isin(pd.date_range('2019-08-13', '2019-09-01'))]
filtered_dfc

In [ ]:
filtered_dfd = a[a["DT"].isin(pd.date_range('2019-09-03', '2019-09-30'))]
filtered_dfd

In [ ]:
filtered_dff = a[a["DT"].isin(pd.date_range('2019-10-28', '2019-11-30'))]
filtered_dff

In [ ]:
# 수익률  (sell 종가- buy 종가)   / buy 종가 * 100

buy_df =masu_df['C_PRC']
sell_df = mado_df['C_PRC']

In [ ]:
buy_df

DT
2019-02-22    6700
2019-04-02    7440
2019-05-15    7220
2019-06-24    6750
2019-08-13    7980
2019-09-03    8420
2019-10-28    9660
Name: C_PRC, dtype: int64

In [ ]:
sell_df
# 13,759,965
# 13.76%

DT
2019-02-28     6200
2019-04-09     6780
2019-05-17     6620
2019-07-01     8400
2019-08-21     7550
2019-09-25     8400
2019-11-08    12100
Name: C_PRC, dtype: int64